In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from random import randint

In [ ]:
!git clone -q https://github.com/coleman-zachery/poker_pytorch.git git_download
!rm -rf libs
!mv git_download/libs .
!rm -rf git_download

from libs.constants import RANKS, SUITS
from libs.classify import classify_poker_hand
from libs.helpers import card_strings_to_tensor, card_str_to_tuple, poker_hand_label_to_index, reverse_poker_hand_index
from libs.encoders import CardEncoder, PokerHandClassifier
from libs.generators import generate_straight_flush, generate_four_of_a_kind

In [1]:
from libs.constants import RANKS, SUITS
from libs.classify import classify_poker_hand

# from libs.helpers import card_strings_to_tensor, card_str_to_tuple, classify_poker_hand, poker_hand_label_to_index, reverse_poker_hand_index
# from libs.encoders import CardEncoder, PokerHandClassifier
# from libs.generators import generate_straight_flush, generate_four_of_a_kind

In [2]:
classify_poker_hand(["A♥", "K♦", "Q♣", "J♠", "__", "K♦", "Q♣", "J♠", "W_"])

[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2], [0, 0, 2, 0, 0, 0], [0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 1, 0]]
0 ([0, 0, 0, 0, 1, 0], [0, 0, 0, 2, 0, 0], [0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0])
1 ([0, 0, 0, 2, 0, 0], [0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0])
2 ([0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0])
3 ([0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0])
4 ([0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0])
5 ([0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0])
6 ([0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]

TypeError: 'int' object is not iterable

In [ ]:
# ---- training set ----
card_strings = [
    ["A♥", "K♦", "Q♣", "J♠", "__"], # high card
    ["6♥", "6♦", "6♣", "6♠", "W?"], # five of a kind
    ["A♥", "K♥", "Q♥", "J♥", "X♥"], # royal flush
    ["9♠", "8♠", "7♠", "6♠", "5♠"], # straight flush
    ["3♦", "3♣", "3♥", "3♠", "9♥"], # four of a kind
    ["4♥", "4♠", "4♣", "9♦", "9♥"], # full house
    ["2♣", "5♣", "9♣", "J♣", "K♣"], # flush
    ["6♥", "5♠", "4♦", "3♣", "2♥"], # straight
    ["7♠", "7♥", "7♣", "2♦", "5♥"], # three of a kind
    ["8♦", "8♣", "4♥", "4♠", "K♥"], # two pair
    ["J♣", "J♠", "3♥", "6♦", "9♣"], # one pair
    ["5♥", "X♠", "7♣", "4♦", "2♥"], # high card
    ["5♥", "5♠", "__", "__", "__"], # one pair with 3 empty cards
    ["9♣", "9♦", "4♥", "4♠", "??"], # two pair and 1 ?? card
    ["__", "__", "__", "__", "__"], # empty hand
]


In [ ]:
# create tensors and labels
card_tensors = []
labels = []
for hand in card_strings:
    card_tensor = card_strings_to_tensor(hand)
    card_tensors.append(card_tensor)
    card_tuples = [card_str_to_tuple(cs) for cs in hand]
    classification = classify_poker_hand(card_tuples)
    print(classification)
    label, card_order = classification
    labels.append(poker_hand_label_to_index(label))

# get classification embeddings
card_tensors = torch.stack(card_tensors)
labels = torch.tensor(labels, dtype=torch.long)
# print("Card Tensors:\n", card_tensors)
# print("Labels:\n", labels)
encoder = CardEncoder()
classifier = PokerHandClassifier() # ---- choose classifier ----
encoded_cards = encoder(card_tensors.view(-1, 2))
encoded_hands = encoded_cards.view(card_tensors.size(0), 5, -1)
outputs = classifier(encoded_hands)
print("Classifier Outputs:\n", outputs)

In [ ]:
# ---- train loop ----
def train_loop(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_cards, batch_labels in data_loader:
            optimizer.zero_grad()
            encoded_cards = encoder(batch_cards.view(-1, 2))
            encoded_hands = encoded_cards.view(batch_cards.size(0), 5, -1)
            outputs = model(encoded_hands)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Example of setting up a data loader and training
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(card_tensors, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(classifier.parameters(), lr=0.001)
train_loop(classifier, data_loader, criterion, optimizer, num_epochs=100)


In [ ]:
# ---- function that outputs the softmax probability for each class >= 1.00%, formatted to .00% and sorted largest to smallest % ----
def get_class_probabilities(model, hand_tensor):
    model.eval()
    with torch.no_grad():
        encoded_cards = encoder(hand_tensor.view(-1, 2))
        encoded_hand = encoded_cards.view(1, 5, -1)
        outputs = model(encoded_hand)
        probabilities = F.softmax(outputs, dim=1).squeeze(0)
        significant_probs = {reverse_poker_hand_index(i): prob.item() for i, prob in enumerate(probabilities)}
        sorted_probs = dict(sorted(significant_probs.items(), key=lambda item: item[1], reverse=True))
        formatted_probs = {k: f"{v*100:.2f}%" for k, v in sorted_probs.items()}
        return formatted_probs

In [ ]:
# ---- print tensor hands, expected output, and classifier outputs ----
for i in range(card_tensors.size(0)):
    hand_tensor = card_tensors[i].unsqueeze(0)
    expected_label = labels[i].item()
    hand_type = reverse_poker_hand_index(expected_label)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    #predicted_label = torch.argmax(outputs, dim=1).item()
    hand = card_strings[i]
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")

# ---- test to see if the model can correctly predict each royal flush hand ----
royal_flush_hands = [
    ["A♣", "K♣", "Q♣", "J♣", "X♣"],
    ["A♦", "K♦", "Q♦", "J♦", "X♦"],
    ["A♥", "K♥", "Q♥", "J♥", "X♥"],
    ["A♠", "K♠", "Q♠", "J♠", "X♠"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    hand_type = "royal flush"
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")


In [ ]:
royal_flush_hands = [
    ["__", "A♣", "K♦", "Q♥", "J?"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"class probabilities: {probabilities}")